In [1]:
import os
import random
import time

import numpy as np
import torch
from absl import app
# from klearn_tcyclone.training_utils.args import FLAGS, ALL_FLAGS
from klearn_tcyclone.training_utils.training_utils import get_default_flag_values
from sklearn.model_selection import train_test_split
from torch import nn
from torch.utils import data

from klearn_tcyclone.climada.tc_tracks import TCTracks
from klearn_tcyclone.data_utils import (
    LinearScaler,
)
from klearn_tcyclone.KNF.modules.eval_metrics import RMSE_TCTracks
from klearn_tcyclone.KNF.modules.models import Koopman
from klearn_tcyclone.KNF.modules.train_utils import (
    eval_epoch_koopman,
    train_epoch_koopman,
)
from klearn_tcyclone.knf_data_utils import TCTrackDataset
from klearn_tcyclone.training_utils.training_utils import set_flags
from absl import app, flags

from klearn_tcyclone.training_utils.training_utils import extend_by_default_flag_values

In [2]:
torch.cuda.is_available()

True

## Import data

Set some specific parameters and load default values for all other parameters.

In [3]:
flag_params = {
    # "seed": 42,
    "year_range": [1980, 1988],
    # "batch_size": 16,
    "num_epochs": 2,
    "train_output_length": 1,
    "input_length": 15
}
flag_params = extend_by_default_flag_values(flag_params)

In [4]:
random.seed(flag_params["seed"])  # python random generator
np.random.seed(flag_params["seed"])  # numpy random generator

torch.manual_seed(flag_params["seed"])
torch.cuda.manual_seed_all(flag_params["seed"])

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


feature_list = [
    "lon",
    "lat",
    "max_sustained_wind",
    "radius_max_wind",
    "radius_oci",
    "central_pressure",
    "environmental_pressure",
]

# these are not contained as flags
# encoder_hidden_dim = flag_params["hidden_dim"]
# decoder_hidden_dim = flag_params["hidden_dim"]
# encoder_num_layers = flag_params["num_layers"]
# decoder_num_layers = flag_params["num_layers"]

output_dim = flag_params["input_dim"]
num_feats = len(feature_list)
learning_rate = flag_params["learning_rate"]
# ---------------

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device", device)

scaler = LinearScaler()
eval_metric = RMSE_TCTracks

Device cuda


In [5]:
flag_params["year_range"]

[1980, 1988]

In [6]:
flag_params["batch_size"]

32

In [7]:
# Datasets
tc_tracks = TCTracks.from_ibtracs_netcdf(
    provider="usa",
    year_range=flag_params["year_range"],
    basin="NA",
    correct_pres=False,
)

tc_tracks_train, tc_tracks_test = train_test_split(tc_tracks.data, test_size=0.1)

2025-01-22 21:02:51,565 - climada.hazard.tc_tracks - WARNING - The cached IBTrACS data set dates from 2023-06-07 23:07:38 (older than 180 days). Very likely, a more recent version is available. Consider manually removing the file /home/ecjb/climada/data/IBTrACS.ALL.v04r00.nc and re-running this function, which will download the most recent version of the IBTrACS data set from the official URL.
2025-01-22 21:03:11,720 - climada.hazard.tc_tracks - WARNING - 49 storm events are discarded because no valid wind/pressure values have been found: 1980199N31284, 1980200N25270, 1980204N23287, 1980226N15339, 1980238N16328, ...


/home/ecjb/packages/climada_python/climada/hazard/tc_tracks.py:614: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  if ibtracs_ds.dims['storm'] == 0:


In [8]:
len(tc_tracks_train), tc_tracks_train[5]

(73,
 <xarray.Dataset> Size: 8kB
 Dimensions:                 (time: 134)
 Coordinates:
   * time                    (time) datetime64[ns] 1kB 1986-08-13T12:00:00 ......
     lat                     (time) float32 536B 30.1 30.46 30.8 ... 56.17 56.2
     lon                     (time) float32 536B -84.0 -84.04 -84.0 ... 6.923 8.0
 Data variables:
     radius_max_wind         (time) float32 536B 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
     radius_oci              (time) float32 536B 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
     max_sustained_wind      (time) float32 536B 10.0 10.0 10.0 ... 15.0 15.0
     central_pressure        (time) float32 536B 1.009e+03 1.01e+03 ... 1.006e+03
     environmental_pressure  (time) float64 1kB 1.01e+03 1.01e+03 ... 1.01e+03
     time_step               (time) float64 1kB 3.0 3.0 3.0 3.0 ... 3.0 3.0 3.0
     basin                   (time) <U2 1kB 'NA' 'NA' 'NA' ... 'NA' 'NA' 'NA'
 Attributes:
     max_sustained_wind_unit:  kn
     central_pressure_unit:    mb
     orig_e

In [9]:

train_set = TCTrackDataset(
    input_length=flag_params["input_length"],
    output_length=flag_params["train_output_length"],
    tc_tracks=tc_tracks_train,
    feature_list=feature_list,
    mode="train",
    jumps=flag_params["jumps"],
    scaler=scaler,
    fit=True,
)
valid_set = TCTrackDataset(
    input_length=flag_params["input_length"],
    output_length=flag_params["train_output_length"],
    tc_tracks=tc_tracks_train,
    feature_list=feature_list,
    mode="valid",
    jumps=flag_params["jumps"],
    scaler=scaler,
    fit=False,
)
test_set = TCTrackDataset(
    input_length=flag_params["input_length"],
    output_length=flag_params["test_output_length"],
    tc_tracks=tc_tracks_test,
    feature_list=feature_list,
    mode="test",
    # jumps=flag_params["jumps"], # jumps not used in test mode
    scaler=scaler,
    fit=False,
)
train_loader = data.DataLoader(
    train_set, batch_size=flag_params["batch_size"], shuffle=True, num_workers=1
)
valid_loader = data.DataLoader(
    valid_set, batch_size=flag_params["batch_size"], shuffle=True, num_workers=1
)
test_loader = data.DataLoader(
    test_set, batch_size=flag_params["batch_size"], shuffle=False, num_workers=1
)

if len(train_loader) == 0:
    raise Exception(
        "There are likely too few data points in the test set. Try to increase year_range."
    )

/home/ecjb/projects/koopman_learning_tropical_cyclone/klearn_tcyclone/data_utils.py:262: RuntimeWarning: divide by zero encountered in divide
  scaling_factor = (self.target_max_vec - self.target_min_vec) / (
/home/ecjb/projects/koopman_learning_tropical_cyclone/klearn_tcyclone/data_utils.py:266: RuntimeWarning: invalid value encountered in multiply
  scaled_diffs_to_min_vec = scaling_factor * diffs_to_min


In [11]:
type(train_loader)

torch.utils.data.dataloader.DataLoader

In [12]:
a = list(train_loader)

In [15]:
n_data = np.sum(
    [
        tc["time"].shape[0] for tc in tc_tracks_train
    ]
)
n_data, n_data**2

2

In [10]:
counter = 0
for inps, tgts in train_loader:
    if counter < 5:
        print(counter)
        print(inps.shape, type(inps))
        print(tgts.shape, type(inps))
        print(inps[0,:,0])
        print(tgts[0,:,0])
        print()
    
    counter += 1


0
torch.Size([32, 15, 7]) <class 'torch.Tensor'>
torch.Size([32, 1, 7]) <class 'torch.Tensor'>
tensor([0.3095, 0.3048, 0.3035, 0.3017, 0.2962, 0.2890, 0.2818, 0.2747, 0.2694,
        0.2637, 0.2553, 0.2462, 0.2387, 0.2304, 0.2198])
tensor([0.2082])

1
torch.Size([32, 15, 7]) <class 'torch.Tensor'>
torch.Size([32, 1, 7]) <class 'torch.Tensor'>
tensor([0.4982, 0.5012, 0.5040, 0.5075, 0.5114, 0.5170, 0.5269, 0.5344, 0.5336,
        0.5281, 0.5215, 0.5139, 0.5078, 0.5012, 0.4928])
tensor([0.4838])

2
torch.Size([32, 15, 7]) <class 'torch.Tensor'>
torch.Size([32, 1, 7]) <class 'torch.Tensor'>
tensor([-0.2573, -0.2615, -0.2652, -0.2682, -0.2700, -0.2705, -0.2700, -0.2692,
        -0.2684, -0.2682, -0.2684, -0.2690, -0.2700, -0.2716, -0.2732])
tensor([-0.2741])

3
torch.Size([32, 15, 7]) <class 'torch.Tensor'>
torch.Size([32, 1, 7]) <class 'torch.Tensor'>
tensor([-0.0236, -0.0372, -0.0521, -0.0673, -0.0817, -0.0958, -0.1101, -0.1243,
        -0.1387, -0.1528, -0.1660, -0.1797, -0.1952, -0.211

In [127]:
"""Pytorch implementation of Koopman Neural Operator."""
import itertools

from klearn_tcyclone.KNF.modules.normalizer import RevIN
import numpy as np
import torch
from torch import nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



class KoopmanKernelSeq2Seq(nn.Module):
    """Koopman Neural Forecaster.

    Attributes:
    input_dim: number of steps of historical observations encoded at every step
    input_length: input length of ts
    output_dim: number of output features
    num_steps: number of prediction steps every forward pass
    encoder_hidden_dim: hidden dimension of encoder
    decoder_hidden_dim: hidden dimension of decoder
    encoder_num_layers: number of layers in the encoder
    decoder_num_layers: number of layers in the decoder
    latent_dim: dimension of finite koopman space num_feats=1: number of
        features
    add_global_operator: whether to use a global operator
    add_control: whether to use a feedback module
    control_num_layers: number of layers in the control module
    control_hidden_dim: hidden dim in the control module
    use_RevIN: whether to use reversible normalization
    use_instancenorm: whether to use instance normalization on hidden states
    regularize_rank: Whether to regularize rank of Koopman Operator.
    num_sins: number of pairs of sine and cosine measurement functions
    num_poly: the highest order of polynomial functions
    num_exp: number of exponential functions
    num_heads: Number of the head the transformer encoder
    transformer_dim: hidden dimension of tranformer encoder
    transformer_num_layers: number of layers in the transformer encoder
    dropout_rate: dropout rate of MLP modules
    """

    def __init__(
        self,
        input_dim,
        input_length,
        output_length,
        output_dim,
        num_steps,
        num_nys_centers,
        rng_seed,
    ):

        super().__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.input_length = input_length
        self.output_length = output_length
        self.num_steps = num_steps
        self.num_nys_centers = num_nys_centers
        self.rng_seed = rng_seed
        self.generator = torch.Generator()
        _ = self.generator.manual_seed(self.rng_seed)        

    def initialize_nystroem_kernels(self, data_set):
        """Initialize the nystroem kernel matrix.

        Args:
        data_set: dataset to initialize the kernel matrix

        Returns:
        nystroem_matrix: nystroem kernel matrix
        """
        rand_indices = self._center_selection(data_set.shape[0], self.generator)



        data_set = torch.tensor(data_set, dtype=torch.float32).to(device)
        nystroem_matrix = torch.mm(data_set, data_set.t())
        return nystroem_matrix

    def _center_selection(self, num_pts: int, generator: torch.Generator | None = None) -> torch.Tensor:
        if self.num_nys_centers < 1:
            num_nys_centers = int(self.num_nys_centers * num_pts)
        else:
            num_nys_centers = int(self.num_nys_centers)

        unif = torch.ones(num_pts)
        rand_indices = unif.multinomial(num_nys_centers, replacement=False, generator=generator)

        return rand_indices

In [128]:
a = KoopmanKernelSeq2Seq(
    1, 1, 1, 1, 1, 10, 42
)

In [129]:
a._center_selection(100, a.generator)

tensor([25, 33, 36, 67, 40, 79, 17, 70, 48, 76])